In [35]:
"""
Extracción de Características de Texto - Análisis de Noticias
Autor: Abraham MD
Fecha: 2025
"""

# === IMPORTACIONES ===
import pathlib
import pandas as pd
import numpy as np
import spacy
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from typing import List, Dict, Tuple, Optional
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

# === CONFIGURACIÓN DE RUTAS ===
BASE_DIR = pathlib.Path.cwd().parent.resolve()
print(f"Directorio base del proyecto: {BASE_DIR}")

# Verificar estructura de directorios



BASE_DIR

Directorio base del proyecto: C:\Users\ABRAHAM\Documents\GitHub\Practica-1


WindowsPath('C:/Users/ABRAHAM/Documents/GitHub/Practica-1')

# Carga y Análisis del Dataset

Este notebook implementa la extracción de características de texto para el análisis de noticias verdaderas vs falsas.

In [15]:
# === CARGA DEL DATASET LIMPIO ===
def load_clean_dataset() -> Optional[pd.DataFrame]:
    """
    Carga el dataset limpio desde múltiples ubicaciones posibles
    
    Returns:
        DataFrame con los datos limpios o None si no se encuentra
    """
    # Posibles ubicaciones del archivo
    possible_paths = [
        BASE_DIR / 'data' / 'corpus_limpio' / 'noticias_combinadas.csv'
    ]
    
    for path in possible_paths:
        try:
            if path.exists():
                print(f"Encontrado dataset en: {path}")
                df = pd.read_csv(path, encoding='utf-8')
                
                
                # Limpiar y preparar datos
                df['text'] = df['text'].astype(str)
                df['label'] = df['label'].astype(bool)
                
                # Eliminar textos vacíos
                initial_len = len(df)
                df = df[df['text'].str.strip() != ''].reset_index(drop=True)
                removed = initial_len - len(df)
                
                if removed > 0:
                    print(f"Eliminados {removed} registros con texto vacío")
                
                print(f"Dataset cargado: {len(df):,} registros")
                print(f"Columnas: {list(df.columns)}")
                print(f"Distribución de etiquetas:")
                label_counts = df['label'].value_counts()
                for label, count in label_counts.items():
                    label_name = "Verdaderas" if label else "Falsas"
                    pct = (count / len(df)) * 100
                    print(f"   • {label_name}: {count:,} ({pct:.1f}%)")
                
                return df
                
        except Exception as e:
            print(f"Error cargando {path}: {e}")
            continue
    
    print("No se pudo cargar el dataset desde ninguna ubicación")
    return None

# Cargar datos
df = load_clean_dataset()
if df is not None:
    print(f"\nVista previa del dataset:")
    display(df.head())
else:
    print("No se puede continuar sin datos")

Encontrado dataset en: C:\Users\ABRAHAM\Documents\GitHub\Practica-1\data\corpus_limpio\noticias_combinadas.csv
Dataset cargado: 5,518 registros
Columnas: ['text', 'label']
Distribución de etiquetas:
   • Verdaderas: 2,839 (51.4%)
   • Falsas: 2,679 (48.6%)

Vista previa del dataset:


,text,label
0,distintas desinformaciones senalan falsamente ...,False
1,el metraje realmente corresponde a embarcacion...,False
2,una cuenta desinformadora de derecha adelanta ...,False
3,se trata de una suplantacion que no ha sido pu...,False
4,un video viral atribuye a hugo “el pollo” carv...,False


In [20]:
# === INICIALIZACIÓN DE SPACY ===
def setup_spacy_model() -> Optional[spacy.Language]:
    """
    Configura y carga el modelo de SpaCy para español
    
    Returns:
        Modelo de SpaCy o None si hay error
    """

    # Intentar cargar el modelo
    nlp = spacy.load("es_core_news_sm")
    
    # Verificar que el modelo funciona
    test_doc = nlp("Esta es una prueba del modelo de SpaCy.")
    if len(test_doc) > 0:
        print("Modelo de SpaCy cargado exitosamente")
        print(f"   • Idioma: {nlp.lang}")
        print(f"   • Pipeline: {nlp.pipe_names}")
        print(f"   • Stopwords disponibles: {len(nlp.Defaults.stop_words)}")
        return nlp
    else:
        print("El modelo no procesa texto correctamente")
        return None
            


# Configurar SpaCy
nlp = setup_spacy_model()

if nlp:
    # Mostrar ejemplo de procesamiento
    sample_text = "Las noticias falsas son un problema grave en la sociedad moderna."
    doc = nlp(sample_text)
    

Modelo de SpaCy cargado exitosamente
   • Idioma: es
   • Pipeline: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
   • Stopwords disponibles: 521


# Tokenización y Stemming

Procesamiento avanzado de texto utilizando SpaCy y NLTK para:
- **Tokenización**: Separar el texto en unidades básicas
- **Stemming**: Reducir palabras a su raíz lexical
- **Filtrado**: Eliminar puntuación y espacios irrelevantes

In [22]:
# === PROCESAMIENTO DE TEXTO: TOKENIZACIÓN Y STEMMING ===

class TextProcessor:
    """
    Clase para procesamiento avanzado de texto con SpaCy y NLTK
    """
    
    def __init__(self, nlp_model=None):
        self.nlp = nlp_model
        self.stemmer = SnowballStemmer('spanish')
        
    def tokenize_and_stem(self, text: str) -> str:
        """
        Tokeniza el texto y aplica stemming usando SpaCy + NLTK
        
        Args:
            text (str): Texto a procesar
            
        Returns:
            str: Texto tokenizado y con stemming aplicado
        """
        if not isinstance(text, str) or not text.strip():
            return ""
        
        if self.nlp is None:
            # Fallback sin SpaCy
            words = text.split()
            stemmed = [self.stemmer.stem(word) for word in words if word.isalpha()]
            return " ".join(stemmed)
        
        try:
            # Procesar con SpaCy
            doc = self.nlp(text)
            
            # Filtrar y aplicar stemming
            processed_tokens = []
            for token in doc:
                # Filtrar tokens no deseados
                if (not token.is_punct and 
                    not token.is_space and 
                    not token.is_digit and
                    len(token.text.strip()) > 1 and
                    token.text.isalpha()):
                    
                    # Aplicar stemming al token limpio
                    stemmed_token = self.stemmer.stem(token.text.lower())
                    processed_tokens.append(stemmed_token)
            
            return " ".join(processed_tokens)
            
        except Exception as e:
            print(f"Error procesando texto: {e}")
            # Fallback simple
            words = text.split()
            return " ".join([self.stemmer.stem(word.lower()) for word in words if word.isalpha()])
    
    def process_dataframe(self, df: pd.DataFrame, text_column: str = 'text') -> pd.DataFrame:
        """
        Procesa una columna de texto en un DataFrame
        
        Args:
            df: DataFrame a procesar
            text_column: Nombre de la columna de texto
            
        Returns:
            DataFrame con columna adicional de texto procesado
        """
        if df.empty or text_column not in df.columns:
            print(f"DataFrame vacío o columna '{text_column}' no encontrada")
            return df
        
        print(f"Procesando {len(df):,} textos...")
        
        # Crear copia del DataFrame
        df_processed = df.copy()
        
        # Aplicar procesamiento
        import time
        start_time = time.time()
        
        df_processed['texto_procesado'] = df_processed[text_column].apply(self.tokenize_and_stem)
        
        end_time = time.time()
        processing_time = end_time - start_time
        
        # Estadísticas
        empty_processed = (df_processed['texto_procesado'] == "").sum()
        avg_tokens_before = df_processed[text_column].str.split().str.len().mean()
        avg_tokens_after = df_processed['texto_procesado'].str.split().str.len().mean()
        
        print(f"Procesamiento completado en {processing_time:.2f} segundos")
        print(f"Estadísticas:")
        print(f"   • Textos vacíos después del procesamiento: {empty_processed}")
        print(f"   • Tokens promedio (antes): {avg_tokens_before:.1f}")
        print(f"   • Tokens promedio (después): {avg_tokens_after:.1f}")
        print(f"   • Reducción de tokens: {((avg_tokens_before - avg_tokens_after) / avg_tokens_before * 100):.1f}%")
        
        return df_processed

# Inicializar procesador
if df is not None:
    processor = TextProcessor(nlp)
    
    # Procesar el dataset
    df_processed = processor.process_dataframe(df, 'text')

    print(f"\nEjemplo de procesamiento:")
    if len(df_processed) > 0:
        sample_idx = 0
        original = df_processed.iloc[sample_idx]['text'][:200]
        processed = df_processed.iloc[sample_idx]['texto_procesado'][:200]
        print(f"Original: {original}...")
        print(f"Procesado: {processed}...")
else:
    print("No hay datos para procesar")

🔤 Procesando 5,518 textos...
✅ Procesamiento completado en 545.00 segundos
📊 Estadísticas:
   • Textos vacíos después del procesamiento: 0
   • Tokens promedio (antes): 182.2
   • Tokens promedio (después): 172.8
   • Reducción de tokens: 5.2%

🔍 Ejemplo de procesamiento:
Original: distintas desinformaciones senalan falsamente la estatua como si fuera del dictador sovietico unas con la imagen original y otras con inteligencia artificial...
Procesado: distint desinform senal fals la estatu com si fuer del dictador soviet unas con la imag original otras con inteligent artificial...
✅ Procesamiento completado en 545.00 segundos
📊 Estadísticas:
   • Textos vacíos después del procesamiento: 0
   • Tokens promedio (antes): 182.2
   • Tokens promedio (después): 172.8
   • Reducción de tokens: 5.2%

🔍 Ejemplo de procesamiento:
Original: distintas desinformaciones senalan falsamente la estatua como si fuera del dictador sovietico unas con la imagen original y otras con inteligencia artificial..

In [23]:
# === VISUALIZACIÓN DEL DATASET PROCESADO ===
if 'df_processed' in locals() and df_processed is not None:
    print("Dataset con texto procesado:")
    display(df_processed[['text', 'texto_procesado', 'label']].head())
    
    # Análisis rápido de longitudes
    if 'texto_procesado' in df_processed.columns:
        longitudes = df_processed['texto_procesado'].str.split().str.len()
        print(f"\nEstadísticas de longitud (tokens procesados):")
        print(f"   • Promedio: {longitudes.mean():.1f} tokens")
        print(f"   • Mediana: {longitudes.median():.1f} tokens")
        print(f"   • Mínimo: {longitudes.min()} tokens")
        print(f"   • Máximo: {longitudes.max()} tokens")
else:
    print("No hay datos procesados para mostrar")

Dataset con texto procesado:


,text,texto_procesado,label
0,distintas desinformaciones senalan falsamente ...,distint desinform senal fals la estatu com si ...,False
1,el metraje realmente corresponde a embarcacion...,el metraj realment correspond embarc zarp tras...,False
2,una cuenta desinformadora de derecha adelanta ...,una cuent desinform de derech adelant dat fals...,False
3,se trata de una suplantacion que no ha sido pu...,se trat de una suplant que no ha sid public po...,False
4,un video viral atribuye a hugo “el pollo” carv...,un vide viral atribu hug el poll carvajal fals...,False



Estadísticas de longitud (tokens procesados):
   • Promedio: 172.8 tokens
   • Mediana: 41.0 tokens
   • Mínimo: 5 tokens
   • Máximo: 4402 tokens


In [ ]:
# === EXTRACCIÓN DE CARACTERÍSTICAS: BAG OF WORDS ===
print("Extrayendo características con Bag of Words...")

class FeatureExtractor:
    """
    Clase para extracción de características de texto
    """
    
    def __init__(self, nlp_model=None):
        self.nlp = nlp_model
        self.stop_words = list(nlp_model.Defaults.stop_words) if nlp_model else None
        
    def extract_bow_features(self, df: pd.DataFrame, text_column: str = 'texto_procesado') -> pd.DataFrame:
        """
        Extrae características usando Bag of Words con diferentes configuraciones
        
        Args:
            df: DataFrame con texto procesado
            text_column: Columna con texto a analizar
            
        Returns:
            DataFrame con características extraídas
        """
        if df.empty or text_column not in df.columns:
            print(f"Error: DataFrame vacío o columna '{text_column}' no encontrada")
            return df
        
        df_features = df.copy()
        
        # 1. BOW básico (unigramas y bigramas)
        print("Extrayendo vocabulario básico (unigramas + bigramas)...")
        vectorizer_basic = CountVectorizer(
            ngram_range=(1, 2),
            max_features=1000,  # Limitar características para eficiencia
            min_df=2,  # Aparecer en al menos 2 documentos
            stop_words=self.stop_words
        )
        
        # Ajustar vectorizador a todos los textos
        all_texts = df_features[text_column].fillna("").tolist()
        vectorizer_basic.fit(all_texts)
        
        print(f"   • Vocabulario básico: {len(vectorizer_basic.vocabulary_):,} términos")
        
        # 2. BOW sin stopwords personalizadas
        print("Extrayendo vocabulario sin stopwords...")
        vectorizer_no_stop = CountVectorizer(
            ngram_range=(1, 2),
            max_features=800,
            min_df=3,
            stop_words=self.stop_words
        )
        vectorizer_no_stop.fit(all_texts)
        
        print(f"   • Vocabulario sin stopwords: {len(vectorizer_no_stop.vocabulary_):,} términos")
        
        # 3. BOW con filtrado por frecuencia
        print("Extrayendo vocabulario filtrado por frecuencia...")
        vectorizer_freq = CountVectorizer(
            ngram_range=(1, 2),
            max_features=600,
            min_df=5,  # Debe aparecer en al menos 5 documentos
            max_df=0.8,  # No más del 80% de documentos
            stop_words=self.stop_words
        )
        vectorizer_freq.fit(all_texts)
        
        print(f"   • Vocabulario filtrado: {len(vectorizer_freq.vocabulary_):,} términos")
        
        # 4. TF-IDF para comparación
        print("Extrayendo características TF-IDF...")
        vectorizer_tfidf = TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=500,
            min_df=3,
            max_df=0.8,
            stop_words=self.stop_words
        )
        vectorizer_tfidf.fit(all_texts)
        
        print(f"   • Vocabulario TF-IDF: {len(vectorizer_tfidf.vocabulary_):,} términos")
        
        # Guardar metadatos de vocabularios para análisis posterior
        df_features['vocab_basico'] = [len(vectorizer_basic.vocabulary_)] * len(df_features)
        df_features['vocab_sin_stopwords'] = [len(vectorizer_no_stop.vocabulary_)] * len(df_features)
        df_features['vocab_filtrado'] = [len(vectorizer_freq.vocabulary_)] * len(df_features)
        df_features['vocab_tfidf'] = [len(vectorizer_tfidf.vocabulary_)] * len(df_features)
        
        # Almacenar vectorizadores para uso posterior
        self.vectorizers = {
            'basico': vectorizer_basic,
            'sin_stopwords': vectorizer_no_stop,
            'filtrado': vectorizer_freq,
            'tfidf': vectorizer_tfidf
        }
        
        return df_features
    
    def analyze_vocabulary_overlap(self) -> None:
        """
        Analiza la superposición entre diferentes vocabularios
        """
        if not hasattr(self, 'vectorizers'):
            print("Primero ejecuta extract_bow_features()")
            return
        
        print("\nAnálisis de superposición de vocabularios:")
        print("-" * 50)
        
        vocabs = {name: set(vec.vocabulary_.keys()) for name, vec in self.vectorizers.items()}
        
        # Calcular intersecciones
        intersections = {}
        for name1, vocab1 in vocabs.items():
            for name2, vocab2 in vocabs.items():
                if name1 != name2:
                    key = f"{name1} ∩ {name2}"
                    intersections[key] = len(vocab1 & vocab2)
        
        # Mostrar resultados
        for key, value in intersections.items():
            print(f"   • {key}: {value:,} términos comunes")
        
        # Términos únicos de cada vocabulario
        print(f"\nTérminos más frecuentes por tipo:")
        for name, vectorizer in self.vectorizers.items():
            if hasattr(vectorizer, 'vocabulary_'):
                # Obtener algunos términos de ejemplo
                sample_terms = list(vectorizer.vocabulary_.keys())[:10]
                print(f"   • {name}: {', '.join(sample_terms[:5])}...")

# Ejecutar extracción de características
if 'df_processed' in locals() and df_processed is not None:
    extractor = FeatureExtractor(nlp)
    df_with_features = extractor.extract_bow_features(df_processed, 'texto_procesado')
    
    # Análizar vocabularios
    extractor.analyze_vocabulary_overlap()
    
    print(f"\nCaracterísticas extraídas para {len(df_with_features):,} documentos")
else:
    print("No hay datos procesados para extraer características")

Extrayendo características con Bag of Words...
Extrayendo vocabulario básico (unigramas + bigramas)...
Extrayendo vocabulario básico (unigramas + bigramas)...
   • Vocabulario básico: 1,000 términos
Extrayendo vocabulario sin stopwords...
   • Vocabulario básico: 1,000 términos
Extrayendo vocabulario sin stopwords...
   • Vocabulario sin stopwords: 800 términos
Extrayendo vocabulario filtrado por frecuencia...
   • Vocabulario sin stopwords: 800 términos
Extrayendo vocabulario filtrado por frecuencia...
   • Vocabulario filtrado: 600 términos
Extrayendo características TF-IDF...
   • Vocabulario filtrado: 600 términos
Extrayendo características TF-IDF...
   • Vocabulario TF-IDF: 500 términos

Análisis de superposición de vocabularios:
--------------------------------------------------
   • basico ∩ sin_stopwords: 800 términos comunes
   • basico ∩ filtrado: 600 términos comunes
   • basico ∩ tfidf: 500 términos comunes
   • sin_stopwords ∩ basico: 800 términos comunes
   • sin_stopword

In [25]:
# === VISUALIZACIÓN DEL DATASET CON CARACTERÍSTICAS ===
if 'df_with_features' in locals() and df_with_features is not None:
    print("Dataset con características extraídas:")
    
    # Mostrar información del dataset
    print(f"Dimensiones: {df_with_features.shape}")
    print(f"Columnas: {list(df_with_features.columns)}")

    # Mostrar vista previa
    display_columns = ['text', 'texto_procesado', 'label', 'vocab_basico', 'vocab_sin_stopwords']
    available_columns = [col for col in display_columns if col in df_with_features.columns]
    
    if available_columns:
        print(f"\nVista previa (columnas: {', '.join(available_columns)}):")
        display(df_with_features[available_columns].head())
    else:
        display(df_with_features.head())
    
    # Resumen estadístico de características
    numeric_cols = df_with_features.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nResumen estadístico de características numéricas:")
        display(df_with_features[numeric_cols].describe())
else:
    print("No hay dataset con características para mostrar")

Dataset con características extraídas:
Dimensiones: (5518, 7)
Columnas: ['text', 'label', 'texto_procesado', 'vocab_basico', 'vocab_sin_stopwords', 'vocab_filtrado', 'vocab_tfidf']

Vista previa (columnas: text, texto_procesado, label, vocab_basico, vocab_sin_stopwords):


,text,texto_procesado,label,vocab_basico,vocab_sin_stopwords
0,distintas desinformaciones senalan falsamente ...,distint desinform senal fals la estatu com si ...,False,1000,800
1,el metraje realmente corresponde a embarcacion...,el metraj realment correspond embarc zarp tras...,False,1000,800
2,una cuenta desinformadora de derecha adelanta ...,una cuent desinform de derech adelant dat fals...,False,1000,800
3,se trata de una suplantacion que no ha sido pu...,se trat de una suplant que no ha sid public po...,False,1000,800
4,un video viral atribuye a hugo “el pollo” carv...,un vide viral atribu hug el poll carvajal fals...,False,1000,800



Resumen estadístico de características numéricas:


,vocab_basico,vocab_sin_stopwords,vocab_filtrado,vocab_tfidf
count,5518.0,5518.0,5518.0,5518.0
mean,1000.0,800.0,600.0,500.0
std,0.0,0.0,0.0,0.0
min,1000.0,800.0,600.0,500.0
25%,1000.0,800.0,600.0,500.0
50%,1000.0,800.0,600.0,500.0
75%,1000.0,800.0,600.0,500.0
max,1000.0,800.0,600.0,500.0


In [26]:
# === EJEMPLO DE VECTORIZACIÓN CON TEXTO DE PRUEBA ===
print("Ejemplo práctico de vectorización...")

def demonstrate_vectorization():
    """
    Demuestra cómo funciona la vectorización con un ejemplo concreto
    """
    # Texto de ejemplo
    sample_texts = [
        "El zorro marrón salta sobre el perro perezoso. El zorro es muy rápido y astuto.",
        "Las noticias falsas se propagan rápidamente en redes sociales modernas.",
        "La inteligencia artificial ayuda a detectar información falsa automáticamente."
    ]
    
    print("Textos de ejemplo:")
    for i, text in enumerate(sample_texts, 1):
        print(f"   {i}. {text}")
    
    if nlp is not None:
        # Usar el vectorizador configurado anteriormente
        if 'extractor' in locals() and hasattr(extractor, 'vectorizers'):
            vectorizer = extractor.vectorizers['basico']
        else:
            # Crear vectorizador simple para demostración
            vectorizer = CountVectorizer(
                ngram_range=(1, 2),
                stop_words=list(nlp.Defaults.stop_words) if nlp else None
            )
            vectorizer.fit(sample_texts)
        
        print(f"\nVocabulario extraído ({len(vectorizer.vocabulary_)} términos):")
        
        # Mostrar algunos términos del vocabulario
        vocab_items = list(vectorizer.vocabulary_.items())
        vocab_items.sort(key=lambda x: x[1])  # Ordenar por índice
        
        print("   Primeros 10 términos:")
        for term, idx in vocab_items[:10]:
            print(f"     '{term}' -> índice {idx}")
        
        # Vectorizar los textos de ejemplo
        print(f"\nMatriz de características (shape: {vectorizer.transform(sample_texts).shape}):")
        feature_matrix = vectorizer.transform(sample_texts).toarray()
        
        # Mostrar matriz con nombres de características
        feature_names = vectorizer.get_feature_names_out()[:10]  # Primeras 10
        print(f"   Características mostradas: {', '.join(feature_names)}")
        print("   Matriz (primeras 10 columnas):")
        for i, row in enumerate(feature_matrix[:, :10]):
            print(f"     Texto {i+1}: {row}")
    else:
        print("SpaCy no disponible, ejemplo limitado")
        
        # Ejemplo básico sin SpaCy
        basic_vectorizer = CountVectorizer(ngram_range=(1, 2))
        basic_vectorizer.fit(sample_texts)

        print(f"Vocabulario básico ({len(basic_vectorizer.vocabulary_)} términos):")
        vocab_sample = list(basic_vectorizer.vocabulary_.keys())[:10]
        print(f"   Muestra: {', '.join(vocab_sample)}")

# Ejecutar demostración
demonstrate_vectorization()

Ejemplo práctico de vectorización...
Textos de ejemplo:
   1. El zorro marrón salta sobre el perro perezoso. El zorro es muy rápido y astuto.
   2. Las noticias falsas se propagan rápidamente en redes sociales modernas.
   3. La inteligencia artificial ayuda a detectar información falsa automáticamente.

Vocabulario extraído (40 términos):
   Primeros 10 términos:
     'artificial' -> índice 0
     'artificial ayuda' -> índice 1
     'astuto' -> índice 2
     'automáticamente' -> índice 3
     'ayuda' -> índice 4
     'ayuda detectar' -> índice 5
     'detectar' -> índice 6
     'detectar información' -> índice 7
     'falsa' -> índice 8
     'falsa automáticamente' -> índice 9

Matriz de características (shape: (3, 40)):
   Características mostradas: artificial, artificial ayuda, astuto, automáticamente, ayuda, ayuda detectar, detectar, detectar información, falsa, falsa automáticamente
   Matriz (primeras 10 columnas):
     Texto 1: [0 0 1 0 0 0 0 0 0 0]
     Texto 2: [0 0 0 0 0 0 0 

In [27]:
# === ANÁLISIS FINAL DEL DATASET PROCESADO ===
if 'df_with_features' in locals() and df_with_features is not None:
    print("RESUMEN FINAL DEL PROCESAMIENTO")
    print("=" * 60)
    
    # Información general
    print(f"Registros totales: {len(df_with_features):,}")
    print(f"Columnas generadas: {len(df_with_features.columns)}")
    
    # Análisis de texto procesado
    if 'texto_procesado' in df_with_features.columns:
        texto_procesado = df_with_features['texto_procesado']
        
        # Estadísticas de longitud
        longitudes = texto_procesado.str.split().str.len()
        print(f"\nAnálisis del texto procesado:")
        print(f"   • Longitud promedio: {longitudes.mean():.1f} tokens")
        print(f"   • Longitud mediana: {longitudes.median():.1f} tokens")
        print(f"   • Rango: {longitudes.min()} - {longitudes.max()} tokens")
        
        # Textos vacíos
        textos_vacios = (texto_procesado.str.strip() == '').sum()
        if textos_vacios > 0:
            print(f"   Textos vacíos después del procesamiento: {textos_vacios}")
        
        # Distribución por etiquetas
        if 'label' in df_with_features.columns:
            print(f"\nDistribución por tipo de noticia:")
            for label in [False, True]:
                subset = df_with_features[df_with_features['label'] == label]
                if len(subset) > 0:
                    avg_length = subset['texto_procesado'].str.split().str.len().mean()
                    label_name = "Falsas" if not label else "Verdaderas"
                    print(f"   • {label_name}: {len(subset):,} textos, {avg_length:.1f} tokens promedio")
    
    # Información de vocabularios
    vocab_cols = [col for col in df_with_features.columns if col.startswith('vocab_')]
    if vocab_cols:
        print(f"\nTamaños de vocabularios extraídos:")
        for col in vocab_cols:
            vocab_size = df_with_features[col].iloc[0] if len(df_with_features) > 0 else 0
            vocab_name = col.replace('vocab_', '').replace('_', ' ').title()
            print(f"   • {vocab_name}: {vocab_size:,} términos")
    
    # Vista final del dataset
    print(f"\nEstructura final del dataset:")
    print(f"Columnas: {list(df_with_features.columns)}")
    
    display(df_with_features.head(3))
    
else:
    print("No hay dataset procesado para mostrar el resumen final")

RESUMEN FINAL DEL PROCESAMIENTO
Registros totales: 5,518
Columnas generadas: 7

Análisis del texto procesado:
Análisis del texto procesado:
   • Longitud promedio: 172.8 tokens
   • Longitud mediana: 41.0 tokens
   • Rango: 5 - 4402 tokens

Distribución por tipo de noticia:

   • Longitud promedio: 172.8 tokens
   • Longitud mediana: 41.0 tokens
   • Rango: 5 - 4402 tokens

Distribución por tipo de noticia:
   • Falsas: 2,679 textos, 131.4 tokens promedio
   • Verdaderas: 2,839 textos, 211.8 tokens promedio

Tamaños de vocabularios extraídos:
   • Basico: 1,000 términos
   • Sin Stopwords: 800 términos
   • Filtrado: 600 términos
   • Tfidf: 500 términos

Estructura final del dataset:
Columnas: ['text', 'label', 'texto_procesado', 'vocab_basico', 'vocab_sin_stopwords', 'vocab_filtrado', 'vocab_tfidf']
   • Falsas: 2,679 textos, 131.4 tokens promedio
   • Verdaderas: 2,839 textos, 211.8 tokens promedio

Tamaños de vocabularios extraídos:
   • Basico: 1,000 términos
   • Sin Stopwords: 8

,text,label,texto_procesado,vocab_basico,vocab_sin_stopwords,vocab_filtrado,vocab_tfidf
0,distintas desinformaciones senalan falsamente ...,False,distint desinform senal fals la estatu com si ...,1000,800,600,500
1,el metraje realmente corresponde a embarcacion...,False,el metraj realment correspond embarc zarp tras...,1000,800,600,500
2,una cuenta desinformadora de derecha adelanta ...,False,una cuent desinform de derech adelant dat fals...,1000,800,600,500


In [34]:
# === EXPORTACIÓN DEL DATASET CON CARACTERÍSTICAS ===
print("Exportando dataset con características extraídas...")

def export_processed_dataset(df: pd.DataFrame, base_dir: pathlib.Path) -> bool:
    """
    Exporta el dataset procesado con manejo robusto de directorios y errores
    
    Args:
        df: DataFrame a exportar
        base_dir: Directorio base del proyecto
        
    Returns:
        bool: True si la exportación fue exitosa
    """
    if df.empty:
        print("Error: DataFrame vacío, no se puede exportar")
        return False
    
    try:
        # Crear directorio de destino
        output_dir = base_dir / 'data' / 'extraccion_caracteristicas'
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Ruta completa del archivo
        output_path = output_dir / 'extraccion_caracteristicas.csv'
        
        # Exportar con manejo de encoding
        df.to_csv(output_path, index=False, encoding='utf-8')
        
        # Verificar exportación
        file_size = output_path.stat().st_size
        file_size_mb = file_size / (1024 * 1024)
        
        print(f"Dataset exportado exitosamente:")
        print(f"   Ubicación: {output_path}")
        print(f"   Registros: {len(df):,}")
        print(f"   Columnas: {len(df.columns)}")
        print(f"   Tamaño: {file_size_mb:.2f} MB")
        
        # Verificar integridad del archivo
        try:
            verification_df = pd.read_csv(output_path, encoding='utf-8', nrows=3)
            if len(verification_df) > 0:
                print("Verificación de integridad: EXITOSA")
            else:
                print("Archivo exportado está vacío")
                return False
        except Exception as verify_error:
            print(f"Error en verificación: {verify_error}")
            return False
        
        # Resumen de columnas exportadas
        print(f"\nColumnas exportadas:")
        for i, col in enumerate(df.columns, 1):
            col_type = df[col].dtype
            non_null = df[col].notna().sum()
            print(f"   {i:2}. {col:<25} ({col_type}) - {non_null:,} valores no nulos")
        
        return True
        
    except PermissionError:
        print("Error: Sin permisos para escribir en el directorio")
        return False
    except Exception as e:
        print(f"Error inesperado durante la exportación: {e}")
        return False

def generate_processing_report(original_df: pd.DataFrame, processed_df: pd.DataFrame) -> None:
    """
    Genera un reporte del procesamiento realizado
    """
    if original_df.empty or processed_df.empty:
        return
    
    print(f"\n{'='*60}")
    print(f"REPORTE DE PROCESAMIENTO DE CARACTERÍSTICAS")
    print(f"{'='*60}")
    
    # Estadísticas de transformación
    original_words = original_df['text'].astype(str).str.split().str.len().sum()
    if 'texto_procesado' in processed_df.columns:
        processed_words = processed_df['texto_procesado'].str.split().str.len().sum()
        word_reduction = ((original_words - processed_words) / original_words) * 100
        print(f"Reducción de palabras: {word_reduction:.1f}%")
        print(f"   • Palabras originales: {original_words:,}")
        print(f"   • Palabras procesadas: {processed_words:,}")
    
    # Características extraídas
    feature_cols = [col for col in processed_df.columns if col.startswith('vocab_')]
    if feature_cols:
        print(f"Características extraídas: {len(feature_cols)} tipos de vocabulario")
    
    # Calidad de datos
    original_empty = (original_df['text'].astype(str).str.strip() == '').sum()
    if 'texto_procesado' in processed_df.columns:
        processed_empty = (processed_df['texto_procesado'].str.strip() == '').sum()
        print(f"Calidad de procesamiento:")
        print(f"   • Textos vacíos iniciales: {original_empty}")
        print(f"   • Textos vacíos finales: {processed_empty}")
    
    print("Procesamiento de características completado exitosamente")

# Ejecutar exportación
if 'df_with_features' in locals() and df_with_features is not None:
    success = export_processed_dataset(df_with_features, BASE_DIR)
    
    if success and 'df' in locals():
        generate_processing_report(df, df_with_features)
else:
    print("No hay dataset con características para exportar")
    print("   Asegúrate de haber ejecutado las celdas anteriores correctamente")

Exportando dataset con características extraídas...
Dataset exportado exitosamente:
   Ubicación: C:\Users\ABRAHAM\Documents\GitHub\Practica-1\data\extraccion_caracteristicas\extraccion_caracteristicas.csv
   Registros: 5,518
   Columnas: 7
   Tamaño: 10.51 MB
Verificación de integridad: EXITOSA

Columnas exportadas:
    1. text                      (object) - 5,518 valores no nulos
    2. label                     (bool) - 5,518 valores no nulos
    3. texto_procesado           (object) - 5,518 valores no nulos
    4. vocab_basico              (int64) - 5,518 valores no nulos
    5. vocab_sin_stopwords       (int64) - 5,518 valores no nulos
    6. vocab_filtrado            (int64) - 5,518 valores no nulos
    7. vocab_tfidf               (int64) - 5,518 valores no nulos

REPORTE DE PROCESAMIENTO DE CARACTERÍSTICAS
Reducción de palabras: 5.2%
   • Palabras originales: 1,005,398
   • Palabras procesadas: 953,324
Características extraídas: 4 tipos de vocabulario
Calidad de procesamiento: